# Calculating PM2.5 exposure and health impacts using https://inmap.run

This notebook goes through some examples of how the EIEIO model hosted at https://inmap.run can be used to calculate population-weighted concentrations (e.g., exposure) and health impacts caused by emissions of PM2.5 and its precursors.

The website exposes a [gRPC](https://grpc.io/) service which we can interact with using a number of different languages. The example here uses Python; other notable options are Go and Javascript.

The available functionality is defined in the `eieio.proto` file which is in the same directory as this notebook, but there is not currently much documentation in that file regarding what the different functions do. Instead, documentation can be found in the server source code [here](https://godoc.org/github.com/spatialmodel/inmap/emissions/slca/eieio).

## Setup

Import the necessary libraries. The first two have been automatically generated from `eieio.proto`, our RPC service definition.

In [1]:
import eieio_pb2_grpc
import eieio_pb2
import grpc
import pandas as pd
import numpy as np

#### Connect to the server at https://inmap.run.

In [2]:
creds = grpc.ssl_channel_credentials()
channel = grpc.secure_channel('inmap.run:443', creds)
stub = eieio_pb2_grpc.EIEIOrpcStub(channel)

## Health impacts

#### Calculate total deaths

In [3]:
totalDeaths = stub.EvaluationHealth(eieio_pb2.EvaluationHealthInput(
    Year = 2015,
    Pollutant = eieio_pb2.TotalPM25,
    Population = "all",
    HR = "NasariACS",
)) # The result here is an array of deaths in each InMAP grid cell.

In [4]:
print("Total Deaths:", np.array(totalDeaths.Data).sum())

Total Deaths: 131002.862263


#### Calculation deaths caused by emitter group

In [6]:
groupDeaths = stub.ProdGroups(eieio_pb2.Selection(
    UseGroup =      "All",
    UseSector =     "All",
    EmitterGroup =  "All",
    EmitterSector = "All",
    ImpactType =       "health",
    Population =       "all",
    FinalDemandType =  eieio_pb2.AllDemand,
    Year =             2014,
    Pollutant =        eieio_pb2.TotalPM25,
))

ValueError: Protocol message Selection has no "UseGroup" field.

In [ ]:
pd.DataFrame({"Group":groupDeaths.Names, "Deaths/year":groupDeaths.Values})

In [ ]:
print("Biogenic and wildfire deaths:", np.array(totalDeaths.Data).sum() - groupDeaths.Values[0])

## Population-weighted concentrations (Exposure)

In [ ]:
demand = stub.FinalDemand(eieio_pb2.FinalDemandInput(
    FinalDemandType = eieio_pb2.AllDemand,
    Year = 2014,
    Location = eieio_pb2.Domestic,
))
def popWtdConc(emitterGroup, population):
    """
        PopWtdConc calculated population-weighted concentrations for the given emitterGroup
        and exposed population. For repeated calls it would be faster to split out the 
        different parts of this function and only call them as necessary, but we combine
        everything here for the sake of simplicity
    """
    emitterMask = None
    if emitterGroup == "All":
        concVec = stub.EvaluationConcentrations(eieio_pb2.EvaluationConcentrationsInput(
            Pollutant = eieio_pb2.TotalPM25,
            Year = 2014,
        ))
    elif emitterGroup == "AllAnthro":
        concVec = stub.Concentrations(eieio_pb2.ConcentrationInput(
            Demand = demand,
            Pollutant = eieio_pb2.TotalPM25,
            Year = 2014,
            Location = eieio_pb2.Domestic,
        ))
    else:
        concVec = stub.Concentrations(eieio_pb2.ConcentrationInput(
            Demand = demand,
            Emitters = stub.EmitterMask(eieio_pb2.StringInput(String = emitterGroup)),
            Pollutant = eieio_pb2.TotalPM25,
            Year = 2014,
            Location = eieio_pb2.Domestic,
        ))
    popIncidence = stub.PopulationIncidence(eieio_pb2.PopulationIncidenceInput(
        Year = 2014,
        Population = population,
        HR = "NasariACS",
    ))
    pop = np.array(popIncidence.Population)
    conc = np.array(concVec.Data)
    return (pop * conc).sum() / pop.sum()

### Total Exposure

In [ ]:
c = popWtdConc("All", "all")
print("Population-weighted concentration (total):", c, "ug/m3")
c = popWtdConc("AllAnthro", "all")
print("Population-weighted concentration (all anthropogenic activity):", c, "ug/m3")

### Exposure by race-ethnicity and source

In [ ]:
# The source groups need to be looked up by these abbreviations rather than the full names.
emitters = ["Ag.", "Coal Elec.", "Cooking", "Const.", "Diesel HD Veh.", "Industrial", "Gas LD Veh.", "Misc.",
                "Non-Coal Elec", "Offroad", "Res. Other", "Res. Gas", "Res. Wood", "Road Dst."]

for pop in stub.Populations(eieio_pb2.Selectors()).Names:
    print(pop)
    df = pd.DataFrame(columns = ["Source", "Exposure"])
    for emitter in emitters:
        df = df.append(pd.DataFrame({"Source": [emitter], "Exposure":[popWtdConc(emitter, pop)]}))
    print(df)

## Geometry

In [ ]:
geom = stub.Geometry(eieio_pb2.GeometryInput(
    SpatialReference = "+proj=lcc +lat_1=33.000000 +lat_2=45.000000 +lat_0=40.000000 +lon_0=-97.000000 +x_0=0 +y_0=0 +a=6370997.000000 +b=6370997.000000 +to_meter=1"
))

In [ ]:
popIncidence = stub.PopulationIncidence(eieio_pb2.PopulationIncidenceInput(
    Year = 2014,
    Population = "all",
    HR = "NasariACS",
))
pop = np.array(popIncidence.Population)

In [ ]:
edgeLength = np.zeros((len(pop)))
for i, r in enumerate(geom.Rectangles):
    edgeLength[i] = r.LR.X - r.LL.X

popWtdEdgeLen = (edgeLength * pop).sum() / pop.sum()
print("Population-weighted edge length =", popWtdEdgeLen/1000, "km")